In [21]:
!pip install hazm
!pip install transformers

In [22]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
import re

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import collections

from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

### Load the data using Pandas

In [23]:
data = pd.read_csv('/kaggle/input/snappfood-persian-sentiment-analysis/Snappfood - Sentiment Analysis.csv', on_bad_lines='skip' , delimiter='\t')
data = data[['comment', 'label', 'label_id']]
data.head()

,comment,label,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0


In [24]:
data.dropna(inplace=True)
data = data.drop_duplicates(subset=['comment'], keep='first')
data = data.reset_index(drop=True)
data['label_id'] = data['label_id'].astype(int)
labels = ['HAPPY', 'SAD']

data.head()

,comment,label,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [25]:
data['comment_len_by_words'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))
data.head(20)

,comment,label,label_id,comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1,9
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0,28
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1,19
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0,21
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0,7
5,بدترین پیتزایی که تا به حال خورده بودم,SAD,1,7
6,از همه لحاظ عالی ممنونم,HAPPY,0,5
7,کیفیت غذا متوسط رو به پایین بود انگار داخل یه ...,SAD,1,25
8,همه اقلام تازه و به روز وخیلیییییی سریع بدستم ...,HAPPY,0,12
9,همه چی خوب ولی هات داگ دورش کلا سوخته بود و دا...,SAD,1,14


In [26]:
min_max_len = data["comment_len_by_words"].min(), data["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 2 	Max: 378


In [27]:
def data_gl_than(data, less_than=150.0, greater_than=3.0, col='comment_len_by_words'):
    data_length = data[col].values
    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])
    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [28]:
data_gl_than(data)

Texts with word length of greater than 3.0 and less than 150.0 includes 99.71% of the whole!


In [29]:
minlim, maxlim = 3, 150

data['comment_len_by_words'] = data['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data = data.dropna(subset=['comment_len_by_words'])
data = data.reset_index(drop=True)

In [30]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=data['comment_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [31]:
fig = go.Figure()

groupby_rate = data.groupby('label_id')['label_id'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_rate.index)),
    y=groupby_rate.tolist(),
    text=groupby_rate.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of rate within comments',
    xaxis_title_text='Rate',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [32]:
data['comment'] = data['comment'].apply(lambda x: re.sub(r'[\da-zA-Z\!\(\)\-\[\]\{\}\;\:\'\"\\\,\<\>\.\/\?\@\#\$\%\^\&\*\_\~\؟\،\٪\×\÷\»\«]', '', x))
data.head(20)

,comment,label,label_id,comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1,9.0
1,قرار بود ساعته برسه ولی نیم ساعت زودتر از موق...,HAPPY,0,28.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...,SAD,1,19.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0,21.0
4,شیرینی وانیلی فقط یک مدل بود,HAPPY,0,7.0
5,بدترین پیتزایی که تا به حال خورده بودم,SAD,1,7.0
6,از همه لحاظ عالی ممنونم,HAPPY,0,5.0
7,کیفیت غذا متوسط رو به پایین بود انگار داخل یه ...,SAD,1,25.0
8,همه اقلام تازه و به روز وخیلیییییی سریع بدستم ...,HAPPY,0,12.0
9,همه چی خوب ولی هات داگ دورش کلا سوخته بود و دا...,SAD,1,14.0


In [33]:
normalizer = hazm.Normalizer()

In [34]:
data['comment'] = data['comment'].apply(lambda x: normalizer.normalize(x.strip()))
data.head(20)

,comment,label,label_id,comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1,9.0
1,قرار بود ساعته برسه ولی نیم ساعت زودتر از موقع...,HAPPY,0,28.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...,SAD,1,19.0
3,عالی بود همه چه درست و به اندازه و کیفیت خوب ا...,HAPPY,0,21.0
4,شیرینی وانیلی فقط یک مدل بود,HAPPY,0,7.0
5,بدترین پیتزایی که تا به حال خورده بودم,SAD,1,7.0
6,از همه لحاظ عالی ممنونم,HAPPY,0,5.0
7,کیفیت غذا متوسط رو به پایین بود انگار داخل یه ...,SAD,1,25.0
8,همه اقلام تازه و به‌روز وخیلیی سریع بدستم رسید...,HAPPY,0,12.0
9,همه چی خوب ولی هات داگ دورش کلا سوخته بود و دا...,SAD,1,14.0


In [35]:
data['cleaned_comment_len_by_words'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))
data['cleaned_comment_len_by_words'] = data['cleaned_comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data.head(20)

,comment,label,label_id,comment_len_by_words,cleaned_comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1,9.0,9.0
1,قرار بود ساعته برسه ولی نیم ساعت زودتر از موقع...,HAPPY,0,28.0,25.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...,SAD,1,19.0,17.0
3,عالی بود همه چه درست و به اندازه و کیفیت خوب ا...,HAPPY,0,21.0,20.0
4,شیرینی وانیلی فقط یک مدل بود,HAPPY,0,7.0,6.0
5,بدترین پیتزایی که تا به حال خورده بودم,SAD,1,7.0,7.0
6,از همه لحاظ عالی ممنونم,HAPPY,0,5.0,5.0
7,کیفیت غذا متوسط رو به پایین بود انگار داخل یه ...,SAD,1,25.0,22.0
8,همه اقلام تازه و به‌روز وخیلیی سریع بدستم رسید...,HAPPY,0,12.0,11.0
9,همه چی خوب ولی هات داگ دورش کلا سوخته بود و دا...,SAD,1,14.0,13.0


In [36]:
data = data.dropna(subset=['cleaned_comment_len_by_words'])
data = data.reset_index(drop=True)

data.head(20)

,comment,label,label_id,comment_len_by_words,cleaned_comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1,9.0,9.0
1,قرار بود ساعته برسه ولی نیم ساعت زودتر از موقع...,HAPPY,0,28.0,25.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...,SAD,1,19.0,17.0
3,عالی بود همه چه درست و به اندازه و کیفیت خوب ا...,HAPPY,0,21.0,20.0
4,شیرینی وانیلی فقط یک مدل بود,HAPPY,0,7.0,6.0
5,بدترین پیتزایی که تا به حال خورده بودم,SAD,1,7.0,7.0
6,از همه لحاظ عالی ممنونم,HAPPY,0,5.0,5.0
7,کیفیت غذا متوسط رو به پایین بود انگار داخل یه ...,SAD,1,25.0,22.0
8,همه اقلام تازه و به‌روز وخیلیی سریع بدستم رسید...,HAPPY,0,12.0,11.0
9,همه چی خوب ولی هات داگ دورش کلا سوخته بود و دا...,SAD,1,14.0,13.0


## Train,Validation,Test split


In [37]:
train, tv = train_test_split(data, test_size=0.20, random_state=1, stratify=data['label_id'])
test, valid = train_test_split(tv, test_size=0.50, random_state=1, stratify=tv['label_id'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['comment'].values.tolist(), train['label_id'].values.tolist()
x_valid, y_valid = valid['comment'].values.tolist(), valid['label_id'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['label_id'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(54992, 5)
(6874, 5)
(6874, 5)


### Configuration

In [38]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.')
else:
    print('CUDA is available!')

device: cuda:0
CUDA is available!


In [39]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [40]:
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'HAPPY': 0, 'SAD': 1}
id2label: {0: 'HAPPY', 1: 'SAD'}


In [41]:
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
config = BertConfig.from_pretrained(MODEL_PATH,
                                    **{
                                        'label2id': label2id,
                                        'id2label': id2label,
                                      }
                                   )

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "HAPPY",
    "1": "SAD"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "HAPPY": 0,
    "SAD": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



### Input Embeddings

In [42]:
idx = np.random.randint(0, len(train))
sample_comment = train.iloc[idx]['comment']
tokens = tokenizer.tokenize(sample_comment)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  Comment: {sample_comment}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  Comment: گوشت قیمه بسیار سفت و غیر قابل خوردن بود
   Tokens: گوشت قیمه بسیار سفت و غیر قابل خوردن بود
Token IDs: [5835, 36666, 3177, 12777, 1379, 3268, 3496, 6146, 2834]


In [43]:
encoding = tokenizer.encode_plus(
    sample_comment,
    max_length=32,
    truncation=True,
    add_special_tokens=True,
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt', 
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    2,  5835, 36666,  3177, 12777,  1379,  3268,  3496,  6146,  2834,
             4,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


### Dataset

In [44]:
class SnappFoodDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, comments, targets=None, max_len=128):
        self.comments = comments
        self.targets = targets

        self.tokenizer = tokenizer
        self.max_len = max_len


    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = str(self.comments[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        inputs['targets'] = torch.tensor(target, dtype=torch.long)
        return inputs


In [45]:
train_data_loader = torch.utils.data.DataLoader(SnappFoodDataset(comments=train['comment'].to_numpy(), targets=train['label_id'].to_numpy(), tokenizer=tokenizer), batch_size=TRAIN_BATCH_SIZE)
valid_data_loader = torch.utils.data.DataLoader(SnappFoodDataset(comments=valid['comment'].to_numpy(), targets=valid['label_id'].to_numpy(), tokenizer=tokenizer), batch_size=VALID_BATCH_SIZE)
test_data_loader = torch.utils.data.DataLoader(SnappFoodDataset(comments=test['comment'].to_numpy(), targets=test['label_id'].to_numpy(), tokenizer=tokenizer), batch_size=TEST_BATCH_SIZE)

In [46]:
class SentimentModel(nn.Module):

    def __init__(self, config):
        super(SentimentModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_PATH, return_dict=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [47]:
pt_model = SentimentModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

pt_model <class '__main__.SentimentModel'>


### Training

In [48]:
optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = (y_true == y_pred).mean()
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss

def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):

            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            _, preds = torch.max(outputs, dim=1)

            loss = loss_fn(outputs, targets)

            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)

    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss

def train_op(model,
             data_loader,
             loss_fn,
             optimizer,
             scheduler,
             step=0,
             print_every_step=100,
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None,
             clip=0.0):

    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        _, preds = torch.max(outputs, dim=1)

        loss = loss_fn(outputs, targets)

        losses.append(loss.item())

        loss.backward()

        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        optimizer.step()

        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)

        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))

            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss

        return eval_loss_min
    return eval_cb

In [53]:
for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model,
        data_loader=train_data_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        step=step,
        print_every_step=1000,
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader)

    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

    eval_y, eval_loss = eval_op(
        model=pt_model,
        data_loader=valid_data_loader,
        loss_fn=loss_fn)

    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/3437 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/430 [00:00<?, ?it/s]

Epoch: 1/3...Step: 1000...Train Loss: 0.355319...Train Acc: 0.846...Valid Loss: 0.318924...Valid Acc: 0.860...
Validation loss decreased (inf --> 0.318924).  Saving model ...


Evaluation... :   0%|          | 0/430 [00:00<?, ?it/s]

...

Evaluation... :   0%|          | 0/430 [00:00<?, ?it/s]

Epoch: 3/3...Step: 9000...Train Loss: 0.113226...Train Acc: 0.960...Valid Loss: 0.472244...Valid Acc: 0.865...


Evaluation... :   0%|          | 0/430 [00:00<?, ?it/s]

Epoch: 3/3...Step: 10000...Train Loss: 0.101460...Train Acc: 0.965...Valid Loss: 0.455955...Valid Acc: 0.861...


Evaluation... :   0%|          | 0/430 [00:00<?, ?it/s]

### Prediction

In [54]:
def predict(model, test_data_loader):
    predictions = []
    prediction_probs = []

    model.eval()
    with torch.no_grad():
        for dl in tqdm(test_data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [55]:
preds, probs = predict(pt_model, test_data_loader)
print(preds.shape, probs.shape)

  0%|          | 0/430 [00:00<?, ?it/s]

(6874,) (6874, 2)


In [56]:
y_test, y_pred = test['label_id'].values, preds
print(classification_report(y_test, y_pred, target_names=labels))

              precision    recall  f1-score   support

       HAPPY       0.87      0.85      0.86      3457
         SAD       0.85      0.88      0.86      3417

    accuracy                           0.86      6874
   macro avg       0.86      0.86      0.86      6874
weighted avg       0.86      0.86      0.86      6874

